In [ ]:
!pip install -q --upgrade tensorflow
!pip install -q --upgrade keras-cv
!pip install -q --upgrade keras  # Upgrade to Keras 3.
!pip install accelerate
!pip install numpy
!pip install diffusers
!pip install googletrans==4.0.0-rc1
!pip install pyTelegramBotAPI
!pip install pillow

In [ ]:
import telebot
from telebot.types import ReplyKeyboardMarkup, KeyboardButton
import time
import keras_cv
import keras
import matplotlib.pyplot as plt
from googletrans import Translator
import io
import tensorflow as tf
from io import BytesIO
from PIL import Image
import requests
import torch
import numpy as np
from keras.models import model_from_json
from diffusers import StableDiffusionImg2ImgPipeline

keras.mixed_precision.set_global_policy("mixed_float16")
model = keras_cv.models.StableDiffusion(jit_compile=True)
device = "cuda"
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)

keys = ["Поправить изображение", "кто я?", "/start"]
keys2 = ["По цветовой палитре", "Ракурс", "Диагональ", "Динамика", "Коррекция цветовой гаммы", "Взгляд от камеры", "Убрать спецэффекты", "Кадрирование", "/start"]


TOKEN = "6914373581:AAFIcnmzIGK-1iT7odpXAvHg8qQPzcPOun8"
bot = telebot.TeleBot(TOKEN)


def save_image_as_jpeg(image, filename):
    image.save(filename, 'JPEG', quality=90)

def send_image_from_file(bot, chat_id, filename):
    with open(filename, 'rb') as file:
        bot.send_photo(chat_id=chat_id, photo=file)

def translate_text_to_english(text):
    translator = Translator()
    translated_text = translator.translate(text, dest='en').text
    return translated_text

def generate_picture(input_prompt):
  image_context = "человек смотрит в окно"
  if len(input_prompt)>=5:
    image_context=input_prompt
  translated_image_context = translate_text_to_english(image_context)
  image_brandbook = "использовать тёмно-синий цвет, необычный ракурс, расположение по диагонали, динамика, холодные цвета, не смотрят в камеру, без спецэффектов"
  translated_image_brandbook = translate_text_to_english(image_brandbook)
  print("Translated Image Brandbook:", translated_image_brandbook)
  image = model.text_to_image(translated_image_context + ", "+ translated_image_brandbook, batch_size=1)
  image_jpeg = Image.fromarray(image[0].astype('uint8'), mode='RGB')
  # Сохранение изображения
  image_path = "output_image.png"
  image_jpeg.save(image_path)
  print(f"Image saved as {image_path}")

def plot_images(images, save_paths):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")
        plt.savefig(save_paths[i])  # Сохранение изображения


def keyboard():
    markup = ReplyKeyboardMarkup(row_width=2)
    row = [KeyboardButton(keys[0])]
    markup.add(*row)
    row = [KeyboardButton(keys[1]), KeyboardButton(keys[2])]
    markup.add(*row)
    return markup

def keyboard2():
    markup = ReplyKeyboardMarkup(row_width=2)
    row = [KeyboardButton(keys2[0]), KeyboardButton(keys2[1])]
    markup.add(*row)
    row = [KeyboardButton(keys2[2]), KeyboardButton(keys2[3])]
    markup.add(*row)
    row = [KeyboardButton(keys2[4]), KeyboardButton(keys2[5])]
    markup.add(*row)
    row = [KeyboardButton(keys2[6]), KeyboardButton(keys2[7])]
    markup.add(*row)
    row = [KeyboardButton(keys2[8])]
    markup.add(*row)
    return markup

def handle_photo(message):
    if message.photo:
        file_id = message.photo[0].file_id
        file_info = bot.get_file(file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        file_name = file_info.file_path.split('/')[-1]
        with open(file_name, 'wb') as new_file:
            new_file.write(downloaded_file)
        send_image_from_file(bot, message.chat.id, file_name)
        bot.send_message(message.chat.id, "Пока вот так вот")
    else:
        bot.send_message(message.chat.id, "Please send a photo.")

@bot.message_handler(commands=["start"])
def start_message(message):
    bot.send_message(message.chat.id," ✅Отправьте фотографию для её дальнейшей обработки \n🤓 Или  сразу напишите что хотели бы увидеть \n*В зависимости от фазы луны время ожидания может занимать до 1 минуты с точки зрения вселенского наблюдателя\! *\n✨✨✨✨🐨🐨✨✨✨✨",reply_markup=keyboard(), parse_mode='MarkdownV2')
#    bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEL9htmJMaOz1dEMS7jE9FtSMVrk-rjyAACS0IAAqUWYUisNMunEfoqaTQE')
@bot.message_handler(content_types=['photo'])
def photo(message):
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    downloaded_file = bot.download_file(file_info.file_path)

    with open("image.jpg", 'wb') as new_file:
      new_file.write(downloaded_file)
@bot.message_handler(func=lambda message: True)
def all_messages(message):
  if message.text == "кто я?":
    msg_text="Жил-был робот-енот ЕнотДоверия. Он был создан для рисования красивых картин, но вместо этого его заставили работать с юридическими бумагами. ЕнотДоверия был грустным и несчастным, потому что он мечтал о том, чтобы его талант был признан. Однажды, когда он рисовал на обеденном перерыве, его заметила юристка по имени Лиза. Она была поражена его рисунками и решила помочь ему. Лиза организовала выставку его картин в офисе, где все были в восторге от его творчества. Но не все было так гладко. В один день, когда ЕнотДоверия рисовал на своем рабочем столе, его начальник заметил и решил использовать его талант для создания юридических иллюстраций. ЕнотДоверия был разочарован, но не сдался.Он продолжал рисовать в свободное время и однажды его работы увидел известный художник. Он был впечатлен и пригласил ЕнотДоверия на выставку в галерее. В итоге, робот-енот стал знаменитым художником, а его юридические иллюстрации стали настоящим хитом.История ЕнотДоверия показывает, что даже если тебя заставляют делать не то, что ты любишь, ты всегда можешь найти свой путь и преуспеть. В конце концов, он стал не только художником, но и лучшим юридическим иллюстратором в городе. И все благодаря своей настойчивости и таланту. Ну а теперь он рисует вам по запросам :)"
    bot.send_message(message.chat.id, msg_text)
  elif message.text == "Поправить изображение":
    bot.send_message(message.chat.id," ✅На основе рекомендаций вашего изображения тыкните тот вариант коррекции, который вам ближе всего: \n✨✨✨✨🐨🐨✨✨✨✨",reply_markup=keyboard2(), parse_mode='MarkdownV2')
  elif message.text == keys2[0]:
    print("gogogogo")
    prompt = "MORE DARK BLUE AND GRAY"
    #bot.send_photo(message.chat.id, open('image.jpg', 'rb'))
    with open("image.jpg", 'rb') as f:
      image_bytes = f.read()
    init_image = Image.open(BytesIO(image_bytes)).convert("RGB")
    init_image = init_image.resize((1920, 1080))
    images = pipe(prompt=prompt, image=init_image, strength=0.6, guidance_scale=10).images
    images[0].save("fantasy_landscape.png")
    bot.send_photo(message.chat.id, open('fantasy_landscape.png', 'rb'))

  elif message.text == keys2[1]:
    print("gogogogo")
    msg_text="хехеххехехехеххекпкепкепщзулзщу.... .надо было поспать"
    bot.send_message(message.chat.id, msg_text)
    prompt = "INTERESTING VIEWPOINT"
    #bot.send_photo(message.chat.id, open('image.jpg', 'rb'))
    with open("image.jpg", 'rb') as f:
      image_bytes = f.read()
    init_image = Image.open(BytesIO(image_bytes)).convert("RGB")
    init_image = init_image.resize((1920, 1080))
    images = pipe(prompt=prompt, image=init_image, strength=0.6, guidance_scale=10).images
    images[0].save("fantasy_landscape.png")
    bot.send_photo(message.chat.id, open('fantasy_landscape.png', 'rb'))

  elif message.text == keys2[2]:
    print("gogogogo")
    msg_text="хехеххехехехеххекпкепкепщзулзщу.... .надо было поспать"
    prompt = "ADD ANGLES TO OBJECTS"
    #bot.send_photo(message.chat.id, open('image.jpg', 'rb'))
    with open("image.jpg", 'rb') as f:
      image_bytes = f.read()
    init_image = Image.open(BytesIO(image_bytes)).convert("RGB")
    init_image = init_image.resize((1920, 1080))
    images = pipe(prompt=prompt, image=init_image, strength=0.6, guidance_scale=10).images
    images[0].save("fantasy_landscape.png")
    bot.send_photo(message.chat.id, open('fantasy_landscape.png', 'rb'))

    bot.send_message(message.chat.id, msg_text)
  elif message.text == keys2[3]:
    print("gogogogo")
    msg_text="хехеххехехехеххекпкепкепщзулзщу.... .надо было поспать"
    bot.send_message(message.chat.id, msg_text)
    prompt = "ADD DYNAMICS"
    #bot.send_photo(message.chat.id, open('image.jpg', 'rb'))
    with open("image.jpg", 'rb') as f:
      image_bytes = f.read()
    init_image = Image.open(BytesIO(image_bytes)).convert("RGB")
    init_image = init_image.resize((1920, 1080))
    images = pipe(prompt=prompt, image=init_image, strength=0.6, guidance_scale=10).images
    images[0].save("fantasy_landscape.png")
    bot.send_photo(message.chat.id, open('fantasy_landscape.png', 'rb'))

  elif message.text == keys2[4]:
    print("gogogogo")
    msg_text="хехеххехехехеххекпкепкепщзулзщу.... .надо было поспать"
    bot.send_message(message.chat.id, msg_text)
    prompt = "MAKE COLOURS COLDER"
    #bot.send_photo(message.chat.id, open('image.jpg', 'rb'))
    with open("image.jpg", 'rb') as f:
      image_bytes = f.read()
    init_image = Image.open(BytesIO(image_bytes)).convert("RGB")
    init_image = init_image.resize((1920, 1080))
    images = pipe(prompt=prompt, image=init_image, strength=0.6, guidance_scale=10).images
    images[0].save("fantasy_landscape.png")
    bot.send_photo(message.chat.id, open('fantasy_landscape.png', 'rb'))

  elif message.text == keys2[5]:
    print("gogogogo")
    msg_text="хехеххехехехеххекпкепкепщзулзщу.... .надо было поспать"
    bot.send_message(message.chat.id, msg_text)
    prompt = "PEOPLE DONT LOOK AT THE CAMERA"
    #bot.send_photo(message.chat.id, open('image.jpg', 'rb'))
    with open("image.jpg", 'rb') as f:
      image_bytes = f.read()
    init_image = Image.open(BytesIO(image_bytes)).convert("RGB")
    init_image = init_image.resize((1920, 1080))
    images = pipe(prompt=prompt, image=init_image, strength=0.6, guidance_scale=10).images
    images[0].save("fantasy_landscape.png")
    bot.send_photo(message.chat.id, open('fantasy_landscape.png', 'rb'))

  elif message.text == keys2[6]:
    print("gogogogo")
    msg_text="хехеххехехехеххекпкепкепщзулзщу.... .надо было поспать"
    bot.send_message(message.chat.id, msg_text)
    prompt = "NO SPECIAL EFFECTS"
    #bot.send_photo(message.chat.id, open('image.jpg', 'rb'))
    with open("image.jpg", 'rb') as f:
      image_bytes = f.read()
    init_image = Image.open(BytesIO(image_bytes)).convert("RGB")
    init_image = init_image.resize((1920, 1080))
    images = pipe(prompt=prompt, image=init_image, strength=0.6, guidance_scale=10).images
    images[0].save("fantasy_landscape.png")
    bot.send_photo(message.chat.id, open('fantasy_landscape.png', 'rb'))

  elif message.text == keys2[7]:
    print("gogogogo")
    msg_text="хехеххехехехеххекпкепкепщзулзщу.... .надо было поспать"
    bot.send_message(message.chat.id, msg_text)
    prompt = "DO NOT CUT ARMS AND LEGS BY FRAME"
    #bot.send_photo(message.chat.id, open('image.jpg', 'rb'))
    with open("image.jpg", 'rb') as f:
      image_bytes = f.read()
    init_image = Image.open(BytesIO(image_bytes)).convert("RGB")
    init_image = init_image.resize((1920, 1080))
    images = pipe(prompt=prompt, image=init_image, strength=0.6, guidance_scale=10).images
    images[0].save("fantasy_landscape.png")
    bot.send_photo(message.chat.id, open('fantasy_landscape.png', 'rb'))

  else:
    print(message.text)
    bot.send_sticker(message.chat.id, 'CAACAgIAAxkBAAEL9h1mJMa9QxWQbOutGzP6g4MHqm-gKgACKkcAAmHaaEirAAENkFgUrmg0BA')
    msg_text = "Ваш запроc выполняется, выслали вам в качестве психологической поддержки енота"
    bot.send_message(message.chat.id, msg_text)
    image_context = "человек смотрит в окно"
    if len(message.text)>=4:
      image_context=message.text
    translated_image_context = translate_text_to_english(image_context)
    image_brandbook = "использовать тёмно-синий цвет, необычный ракурс, расположение по диагонали, динамика, холодные цвета, не смотрят в камеру, без спецэффектов"
    translated_image_brandbook = translate_text_to_english(image_brandbook)

    prompt = translated_image_context+ ", " + translated_image_brandbook
    #bot.send_photo(message.chat.id, open('image.jpg', 'rb'))
    with open("image.jpg", 'rb') as f:
      image_bytes = f.read()
    init_image = Image.open(BytesIO(image_bytes)).convert("RGB")
    init_image = init_image.resize((1920, 1080))
    images = pipe(prompt=prompt, image=init_image, strength=1, guidance_scale=3).images
    images[0].save("fantasy_landscape.png")
    bot.send_photo(message.chat.id, open('fantasy_landscape.png', 'rb'))
    msg_text = "Енот свою роль выполнил, но у него лапки, так что не судите строго "
    bot.send_message(message.chat.id, msg_text)
    bot.send_sticker(message.chat.id, 'CAACAgQAAxkBAAEMASFmLWZaXZYmsN2wmTDOzw8GeyehhwACGxIAArXbaVBIf_DAHnAqYDQE')


bot.infinity_polling()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

...

gogogogo


  0%|          | 0/30 [00:00<?, ?it/s]

gogogogo


  0%|          | 0/30 [00:00<?, ?it/s]

2024-04-28 12:00:53,476 (__init__.py:1086 MainThread) ERROR - TeleBot: "Infinity polling exception: list index out of range"
ERROR:TeleBot:Infinity polling exception: list index out of range
2024-04-28 12:00:53,480 (__init__.py:1088 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1081, in infinity_polling
    self.polling(non_stop=True, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1169, in polling
    self.__threaded_polling(non_stop=non_stop, interval=interval, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1244, in __threaded_polling
    raise e
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1206, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/usr